In [1]:
import zipfile
import pandas as pd
import numpy as np
import os
file_path = "C:/Users/Dell/Desktop/Total.csv"

In [2]:
def process_salary_data(file_path):
    # Load the data
    salary_data = pd.read_csv(file_path)
    
    # Define numeric columns
    numeric_columns = ['BasePay', 'OvertimePay', 'OtherPay', 'Benefits', 'TotalPay', 'TotalPayBenefits']
    
    # Function to convert to float, replacing non-convertible values with NaN
    def safe_float(x):
        try:
            return float(x)
        except (ValueError, TypeError):
            return np.nan
    
    # Apply safe conversion to numeric columns
    for col in numeric_columns:
        salary_data[col] = salary_data[col].apply(safe_float)
    
    # Group by JobTitle and aggregate by mean, ignoring non-numeric values
    salary_grouped = salary_data.groupby('JobTitle')[numeric_columns].mean()
    
    # Convert to dictionary
    employee_dict = salary_grouped.to_dict('index')
    
    return salary_data, employee_dict

In [3]:
def get_employee_details_safe(job_title, employee_dict):
    try:
        if job_title not in employee_dict:
            raise ValueError(f"No employee found with the job title: {job_title}")
        return employee_dict[job_title]
    except Exception as e:
        return f"An error occurred: {str(e)}"

In [6]:
def get_employee_details(name, salary_data):
    try:
        employee = salary_data[salary_data['EmployeeName'].str.lower() == name.lower()]
        if employee.empty:
            return f"No employee found with the name: {name}"
        else:
            return employee.to_dict('records')[0]
    except Exception as e:
        return f"An error occurred: {str(e)}"

In [20]:
def export_employee_to_csv(name, salary_data, directory='Employee Profile'):
    try:
        details = get_employee_details(name, salary_data)
        if isinstance(details, str):
            return details

        # Get the full path of the current working directory
        current_dir = os.getcwd()

        # Create the full path for the new directory
        full_directory_path = os.path.join(current_dir, directory)

        # Create the directory if it doesn't exist
        if not os.path.exists(full_directory_path):
            os.makedirs(full_directory_path)
        
        print(f"Created directory: {full_directory_path}")

        # Define file path
        file_path = os.path.join(full_directory_path, f'{name}_details.csv')

       # Save to CSV
        df = pd.DataFrame([details])
        df.to_csv(file_path, index=False)
        
        print(f"Exported CSV file: {file_path}")

        # Create a zip file
        zip_path = os.path.join(full_directory_path, 'Employee Profile.zip')
        with zipfile.ZipFile(zip_path, 'w') as zipf:
            zipf.write(file_path, os.path.basename(file_path))

        print(f"Created zip file: {zip_path}")

        return f"Exported and zipped: {zip_path}"
    except Exception as e:
        return f"An error occurred: {str(e)}"

In [21]:
# Main execution
if __name__ == "__main__":
    file_path = "C:/Users/Dell/Desktop/Total.csv"  
    salary_data, employee_dict = process_salary_data(file_path)

    print(get_employee_details_safe("GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY", employee_dict))
    print(get_employee_details("NATHANIEL FORD", salary_data))
    print(export_employee_to_csv("NATHANIEL FORD", salary_data))

C:\Users\Dell\AppData\Local\Temp\ipykernel_12256\834520004.py:3: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  salary_data = pd.read_csv(file_path)


{'BasePay': 199119.15, 'OvertimePay': 0.0, 'OtherPay': 200092.125, 'Benefits': nan, 'TotalPay': 399211.275, 'TotalPayBenefits': 399211.275}
{'EmployeeName': 'NATHANIEL FORD', 'JobTitle': 'GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY', 'BasePay': 167411.18, 'OvertimePay': 0.0, 'OtherPay': 400184.25, 'Benefits': nan, 'TotalPay': 567595.43, 'TotalPayBenefits': 567595.43, 'Year': 2011}
Created directory: C:\Users\Dell\Employee Profile
Exported CSV file: C:\Users\Dell\Employee Profile\NATHANIEL FORD_details.csv
Created zip file: C:\Users\Dell\Employee Profile\Employee Profile.zip
Exported and zipped: C:\Users\Dell\Employee Profile\Employee Profile.zip
